In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from opendata import OpenData
import pandas as pd
from plotly.offline import iplot, init_notebook_mode
from plotly import graph_objs as go
from matplotlib import pyplot as plt
import maya
from scipy.optimize import curve_fit
import numpy as np
from stravaio import StravaIO
import glob
import os
import json
from pprint import pprint
import plotly_express as px
import sweat
from ca import cp_fit, robust_max
from loguru import logger

init_notebook_mode(connected=True)

# List all local athletes (open data)

In [29]:
od = OpenData()

In [30]:
atl = od.local_athletes()

In [31]:
athletes = []
for a in atl:
    cp = pd.Series([v['METRICS'].get('cp_setting', None) for v in a.metadata['RIDES'].values()])
    n_cp = len(cp.dropna().unique())
    logger.info(f"Athlete {a.id} got {n_cp} unique values")
    if n_cp > 30:
        athletes.append(a)

2019-05-12 06:54:11.908 | INFO     | __main__:<module>:5 - Athlete afb6b45a-bea6-4450-8727-0c105ecd1463 got 3 unique values
2019-05-12 06:54:12.216 | INFO     | __main__:<module>:5 - Athlete 74b3855c-b812-4676-9e37-2e88fc51f523 got 1 unique values
2019-05-12 06:54:13.111 | INFO     | __main__:<module>:5 - Athlete 04dbb597-fc85-4ace-8e4c-f749d3762c5f got 1 unique values
2019-05-12 06:54:14.760 | INFO     | __main__:<module>:5 - Athlete f6de9502-3834-4f42-9e69-2f65bb40e40e got 4 unique values
2019-05-12 06:54:15.335 | INFO     | __main__:<module>:5 - Athlete f4fdb967-368d-46ef-98f0-eb4044be7613 got 1 unique values
2019-05-12 06:54:15.867 | INFO     | __main__:<module>:5 - Athlete 6ca67b8a-d3ed-4a1a-86a0-511ed3e80398 got 6 unique values
2019-05-12 06:54:16.359 | INFO     | __main__:<module>:5 - Athlete bbafe8e9-e75f-41bf-8ddc-622ba4896026 got 1 unique values
2019-05-12 06:54:17.953 | INFO     | __main__:<module>:5 - Athlete 8597fa79-938f-4796-b9d8-4d3e326d86bb got 1 unique values
2019-05-

2019-05-12 06:55:26.056 | INFO     | __main__:<module>:5 - Athlete 2ebc0824-d493-4243-8bb5-dd76ff1dd77c got 1 unique values
2019-05-12 06:55:26.991 | INFO     | __main__:<module>:5 - Athlete b8101019-6f71-42f3-ac72-b5b6fbc3bd2e got 1 unique values
2019-05-12 06:55:27.741 | INFO     | __main__:<module>:5 - Athlete 3b06d93d-dcc6-47e2-9f96-41138cf245b5 got 1 unique values
2019-05-12 06:55:29.629 | INFO     | __main__:<module>:5 - Athlete 1e0a8db8-ed03-4bef-acbc-2c68944a3671 got 8 unique values
2019-05-12 06:55:30.943 | INFO     | __main__:<module>:5 - Athlete 5aaf53a2-ec69-419d-b79f-b896148e1eb7 got 8 unique values
2019-05-12 06:55:31.546 | INFO     | __main__:<module>:5 - Athlete c7a164f3-c025-42bb-af9c-6c1943d06636 got 11 unique values
2019-05-12 06:55:32.370 | INFO     | __main__:<module>:5 - Athlete 43f6be61-d990-445d-b140-afbe936d0696 got 1 unique values
2019-05-12 06:55:32.678 | INFO     | __main__:<module>:5 - Athlete c131b8ea-a19a-495c-a82d-ea56b84495e9 got 4 unique values
2019-05

2019-05-12 06:56:27.077 | INFO     | __main__:<module>:5 - Athlete c951a4f7-b06e-4743-8ad0-24271b8f7cd6 got 2 unique values
2019-05-12 06:56:27.775 | INFO     | __main__:<module>:5 - Athlete 6b11e3c7-fa87-4425-9d84-c3678cc7a8e0 got 1 unique values
2019-05-12 06:56:28.346 | INFO     | __main__:<module>:5 - Athlete a00576d4-2973-4f6c-abeb-395bdb0731f5 got 1 unique values
2019-05-12 06:56:31.100 | INFO     | __main__:<module>:5 - Athlete 62752551-b668-4bba-8d50-48bebc4ddf15 got 16 unique values
2019-05-12 06:56:31.883 | INFO     | __main__:<module>:5 - Athlete 12fcb8e7-f2f4-41ae-ba0c-7a488ae43eba got 7 unique values
2019-05-12 06:56:32.449 | INFO     | __main__:<module>:5 - Athlete b413bada-8941-4ab7-a314-28dd4f5f123b got 1 unique values
2019-05-12 06:56:32.955 | INFO     | __main__:<module>:5 - Athlete 5d552446-8629-4462-8ef8-1cec1fa0964a got 1 unique values
2019-05-12 06:56:33.166 | INFO     | __main__:<module>:5 - Athlete bbe91827-faf7-45f9-a696-e23c837c1552 got 1 unique values
2019-05

2019-05-12 06:57:36.312 | INFO     | __main__:<module>:5 - Athlete cf70e2a1-ad11-48dd-b836-265fadd0fcc2 got 12 unique values
2019-05-12 06:57:37.487 | INFO     | __main__:<module>:5 - Athlete 3516c544-ac97-4ef3-a93b-b669182eb3db got 1 unique values
2019-05-12 06:57:41.907 | INFO     | __main__:<module>:5 - Athlete 3a2073f0-c882-46b5-a907-f28b67246e78 got 6 unique values
2019-05-12 06:57:42.573 | INFO     | __main__:<module>:5 - Athlete fa451263-f0a7-4bb2-82af-37e46972158c got 3 unique values
2019-05-12 06:57:44.026 | INFO     | __main__:<module>:5 - Athlete 8dcc998c-09a3-4496-9872-a8d1390cc191 got 1 unique values
2019-05-12 06:57:45.104 | INFO     | __main__:<module>:5 - Athlete 40f04f0b-07b7-4f6a-917a-9c8aa0ea36f4 got 14 unique values
2019-05-12 06:57:45.527 | INFO     | __main__:<module>:5 - Athlete 1cfdf6b8-568b-4b01-a283-1e1dc1ba7e65 got 4 unique values
2019-05-12 06:57:45.722 | INFO     | __main__:<module>:5 - Athlete 7b488c4e-bdb9-4d49-9561-05804c8140c7 got 1 unique values
2019-0

2019-05-12 06:58:49.327 | INFO     | __main__:<module>:5 - Athlete 7eeda935-23c4-4178-8b01-8592988019fa got 11 unique values
2019-05-12 06:58:50.967 | INFO     | __main__:<module>:5 - Athlete 7567ac31-d479-4b25-9538-541ae8085288 got 20 unique values
2019-05-12 06:58:52.066 | INFO     | __main__:<module>:5 - Athlete f6bbc3cc-d4de-4fd4-895a-e2dc41b95a38 got 1 unique values
2019-05-12 06:58:52.471 | INFO     | __main__:<module>:5 - Athlete 901932c9-b7d2-4ca3-baaf-cbbcc48cfefa got 3 unique values
2019-05-12 06:58:53.449 | INFO     | __main__:<module>:5 - Athlete dfd535f3-b7a5-4350-a464-2efb021845ab got 1 unique values
2019-05-12 06:58:54.482 | INFO     | __main__:<module>:5 - Athlete df5c1f03-dba4-44c6-bf34-0daaac7f64ed got 1 unique values
2019-05-12 06:58:55.928 | INFO     | __main__:<module>:5 - Athlete b5728df5-8b98-4857-b293-1a3a9e329f8c got 8 unique values
2019-05-12 06:58:57.262 | INFO     | __main__:<module>:5 - Athlete 25c9e7b6-d645-4068-9774-dcc29848b758 got 1 unique values
2019-0

...

2019-05-12 07:05:35.565 | INFO     | __main__:<module>:5 - Athlete 6126f117-b584-4569-bacb-d1ce2adb6fb1 got 15 unique values
2019-05-12 07:05:36.085 | INFO     | __main__:<module>:5 - Athlete c78b4183-022e-4efa-9300-85e7c5499c85 got 4 unique values
2019-05-12 07:05:36.521 | INFO     | __main__:<module>:5 - Athlete 0c403203-98e0-49ce-bed7-7d95171ac612 got 2 unique values
2019-05-12 07:05:37.498 | INFO     | __main__:<module>:5 - Athlete b705fb64-70e1-4fe8-adea-c7cd9a3ba6ba got 1 unique values
2019-05-12 07:05:37.998 | INFO     | __main__:<module>:5 - Athlete fe89ed7b-fff3-4550-b520-88ba0bca2b23 got 1 unique values
2019-05-12 07:05:38.072 | INFO     | __main__:<module>:5 - Athlete 46d148ae-9324-4a36-bbf2-70408f4828d3 got 1 unique values
2019-05-12 07:05:38.518 | INFO     | __main__:<module>:5 - Athlete 5381e242-2dfa-4a06-a33d-589394cd56a8 got 4 unique values
2019-05-12 07:05:39.322 | INFO     | __main__:<module>:5 - Athlete b178f464-a771-4b54-9ed0-82c374c9210d got 4 unique values
2019-05

2019-05-12 07:06:31.665 | INFO     | __main__:<module>:5 - Athlete 10a84105-6024-4016-8942-5c920e53e9ef got 2 unique values
2019-05-12 07:06:32.395 | INFO     | __main__:<module>:5 - Athlete d5dccc49-5894-4957-a0e9-addaf73c85d7 got 23 unique values
2019-05-12 07:06:33.280 | INFO     | __main__:<module>:5 - Athlete 25b84440-5a3c-4069-a0c8-b25238789059 got 6 unique values
2019-05-12 07:06:34.596 | INFO     | __main__:<module>:5 - Athlete 9e21b009-78b9-42e7-b598-93a2d4b31e25 got 1 unique values
2019-05-12 07:06:36.365 | INFO     | __main__:<module>:5 - Athlete d096c44f-a7b7-4549-9931-038d42d62973 got 1 unique values
2019-05-12 07:06:36.977 | INFO     | __main__:<module>:5 - Athlete e256046b-ed3f-4daa-9ebd-7021cf036c6b got 1 unique values
2019-05-12 07:06:37.410 | INFO     | __main__:<module>:5 - Athlete effc420f-2e00-4d6a-91bd-036df6a5bd9f got 1 unique values
2019-05-12 07:06:39.433 | INFO     | __main__:<module>:5 - Athlete 21daa2e5-a085-40db-88d8-db1fefb4a6f8 got 7 unique values
2019-05

2019-05-12 07:07:41.434 | INFO     | __main__:<module>:5 - Athlete 7ad25200-694b-49f3-a978-1ac9d8f4b22f got 2 unique values
2019-05-12 07:07:42.307 | INFO     | __main__:<module>:5 - Athlete 3df424f4-43c6-43c0-9c73-9cb11a0dd59e got 1 unique values
2019-05-12 07:07:43.093 | INFO     | __main__:<module>:5 - Athlete bdd05185-fb76-42af-8055-9aefa0dd3af6 got 2 unique values
2019-05-12 07:07:43.937 | INFO     | __main__:<module>:5 - Athlete 2ab26e13-a6cb-4f83-8cbd-54cc41c8025d got 1 unique values
2019-05-12 07:07:44.505 | INFO     | __main__:<module>:5 - Athlete 5de849a1-50d6-4967-89f8-324544cb8e76 got 23 unique values
2019-05-12 07:07:45.497 | INFO     | __main__:<module>:5 - Athlete da1b014b-8a6a-48e3-b7b1-be9779608348 got 1 unique values
2019-05-12 07:07:49.340 | INFO     | __main__:<module>:5 - Athlete 9524c2b5-98af-4b15-b3d5-cb9ffe95a4b9 got 14 unique values
2019-05-12 07:07:51.975 | INFO     | __main__:<module>:5 - Athlete 11e4cbe3-3017-494f-9c08-0abcf67f2b40 got 1 unique values
2019-0

2019-05-12 07:08:48.184 | INFO     | __main__:<module>:5 - Athlete 5e431910-acdb-48e8-9522-5abf0945d680 got 1 unique values
2019-05-12 07:08:49.097 | INFO     | __main__:<module>:5 - Athlete 706aa871-617c-46bb-a513-8f627e017bff got 5 unique values
2019-05-12 07:08:49.619 | INFO     | __main__:<module>:5 - Athlete 617b3c1f-8669-4177-9817-775dbee367f9 got 20 unique values
2019-05-12 07:08:50.190 | INFO     | __main__:<module>:5 - Athlete fb0d270b-f07d-4207-832a-bb71a2868d75 got 3 unique values
2019-05-12 07:08:50.293 | INFO     | __main__:<module>:5 - Athlete b7326e91-61e1-4e56-9f94-5dc1dc80228c got 1 unique values
2019-05-12 07:08:52.766 | INFO     | __main__:<module>:5 - Athlete 2721d67b-a1c9-403c-b74e-f3a2f9bfb9e8 got 12 unique values
2019-05-12 07:08:53.579 | INFO     | __main__:<module>:5 - Athlete a88a5de9-fc43-401d-8fd2-ebad3fbe69b7 got 6 unique values
2019-05-12 07:08:55.154 | INFO     | __main__:<module>:5 - Athlete ac06fd8c-b873-4f47-bf92-b7cec1baa25a got 1 unique values
2019-0

2019-05-12 07:09:59.885 | INFO     | __main__:<module>:5 - Athlete dd85756c-5c33-400b-8142-76ff7008d32e got 13 unique values
2019-05-12 07:10:00.446 | INFO     | __main__:<module>:5 - Athlete cc6bfd83-12d7-4caf-a76b-3fa646955c87 got 3 unique values
2019-05-12 07:10:01.008 | INFO     | __main__:<module>:5 - Athlete 3c3e47ca-7c5d-4a84-a29a-d0294f3a9c54 got 1 unique values
2019-05-12 07:10:02.647 | INFO     | __main__:<module>:5 - Athlete 22526d15-04b5-48f1-aba2-9fc7e3b0491a got 22 unique values
2019-05-12 07:10:04.373 | INFO     | __main__:<module>:5 - Athlete e656449e-823b-44b0-b53d-80f732facb52 got 7 unique values
2019-05-12 07:10:06.003 | INFO     | __main__:<module>:5 - Athlete 09d9fc91-5a24-409d-ba41-5ecb809ad65f got 13 unique values
2019-05-12 07:10:07.003 | INFO     | __main__:<module>:5 - Athlete ad6e148d-2013-4290-80a3-dd77034e55cd got 10 unique values
2019-05-12 07:10:07.554 | INFO     | __main__:<module>:5 - Athlete 415a05b3-ef4d-4733-8f13-7211857935b0 got 19 unique values
201

In [32]:
len(athletes)

5

In [159]:
[print(a.id) for a in athletes]

4a0c275c-039a-415a-89e7-6c34a3db0c49
eefb9a06-0c66-4de5-8115-26124b86b9a0
06f81197-504a-478b-a347-052d4df03044
a36decc0-bbec-4f46-9344-511af06ea02a
cf6fbbc6-1149-4896-a4af-5a88f16283b6


[None, None, None, None, None]

# Extract all bike rides and create a list of dict

In [33]:
a = athletes[0]

In [34]:
rides = []

for k, v in a.metadata['RIDES'].items():
    if v['sport'] == 'Bike':
        _d = {
            'id': a.id,
            'date': maya.parse(v.get('date')).iso8601(),
            'workout_time': v['METRICS'].get('workout_time'),
            'coggan_if': v['METRICS'].get('coggan_if', [0, 0])[0],
            '1m_critical_power': v['METRICS'].get('1m_critical_power'),
            '2m_critical_power': v['METRICS'].get('2m_critical_power'),
            '3m_critical_power': v['METRICS'].get('3m_critical_power'),
            '5m_critical_power': v['METRICS'].get('5m_critical_power'),
            '8m_critical_power': v['METRICS'].get('8m_critical_power'),
            '10m_critical_power': v['METRICS'].get('10m_critical_power'),
            '20m_critical_power': v['METRICS'].get('20m_critical_power'),
            '30m_critical_power': v['METRICS'].get('30m_critical_power'),
            '60m_critical_power': v['METRICS'].get('60m_critical_power'),
            '20m_peak_wpk': v['METRICS'].get('20m_peak_wpk'),
            'cp_setting': v['METRICS'].get('cp_setting')
        }
        rides.append(_d)
logger.info(f"Found {len(rides)} rides")

2019-05-12 07:52:59.598 | INFO     | __main__:<module>:23 - Found 2017 rides


# Create a dataframe of rides and perform data type conversion

In [105]:
df = pd.DataFrame(rides)
col_to_num = ['workout_time', '20m_peak_wpk', 'cp_setting', 'coggan_if',
              '1m_critical_power', '2m_critical_power', '3m_critical_power', '5m_critical_power', '8m_critical_power',
              '10m_critical_power', '20m_critical_power', '30m_critical_power', '60m_critical_power']
df[col_to_num] = df[col_to_num].astype('float')
df['date'] = pd.to_datetime(df['date'])
df = df.dropna().reset_index(drop=True)
df = df.set_index('date', drop=False)
logger.info(f"Size of the df: {len(df)}")

2019-05-12 11:14:23.913 | INFO     | __main__:<module>:9 - Size of the df: 1838


In [106]:
trace1 = go.Scatter(x=df.date, y=df.cp_setting, mode='lines', name='CP Setting')
trace2 = go.Scatter(x=df.date, y=df['20m_critical_power'], mode='markers', name='20m Power')
iplot(go.Figure([trace1, trace2]))

In [158]:
trace1 = go.Scatter(x=df.date, y=df['20m_critical_power'].rolling('42D').max(), fill='tozeroy', mode='none', name='20m')
trace2 = go.Scatter(x=df.date, y=df['5m_critical_power'].rolling('42D').max(), fill='tonexty', mode='none', name='5m')
trace3 = go.Scatter(x=df.date, y=df['1m_critical_power'].rolling('42D').max(), fill='tonexty', mode='none', name='1m')
trace4 = go.Scatter(x=df.date, y=df.cp_setting, name='CP Setting')


layout = go.Layout(
    yaxis=go.layout.YAxis(
        title='Power [Watts]',
    ),
    legend=dict(orientation='h', x=0.3, y=1.1)
)

fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)

iplot(fig)



# Estimate CP

In [36]:
def estimate_cp(df):
    """Estimate rolling cp by:
        1. Apllying rolling best_max across 7 times
        2. Performing CP fit on the results
        3. Applying rolling on the CP
    """
    QNTL = 0.99
    ROL = '42D'
    CP_COLS = ['1m_critical_power', '2m_critical_power', '3m_critical_power', '5m_critical_power', '8m_critical_power',
              '10m_critical_power', '20m_critical_power']
    CP_COLS_ROLLING = [s + '_rolling' for s in CP_COLS]
    _df = df
    _df = _df.set_index('date', drop=False).sort_index()
    _df[CP_COLS_ROLLING] = _df[CP_COLS].rolling(ROL, min_periods=1, axis=0).apply(robust_max, raw=False)
    _df['cp'] = _df[CP_COLS_ROLLING].apply(cp_fit, axis=1)
    _df['cp_rolling'] = _df['cp'].rolling(ROL, min_periods=1, axis=0).max()
    return _df

# Plot data

In [37]:
def plot_data(_df):
    
    cp_rolling = go.Scatter({
        'name': '2d rolling 42D CP from fit',
        'x': _df['date'],
        'y': _df['cp_rolling'],
        'mode': 'markers',
        'marker': {
            'color': 'navy'
        }
    })

    cp_setting = go.Scatter({
        'name': 'Athlete CP settings',
        'x': _df['date'],
        'y': _df['cp_setting']
    })

    data = [cp_rolling, cp_setting]
    
    for col in ['1m_critical_power_rolling', '5m_critical_power_rolling', '20m_critical_power_rolling']:
        _trace = go.Scatter({
            'name': col,
            'x': _df['date'],
            'y': _df[col],
            'mode': 'markers'
        })
        data.append(_trace)
    
    return data

layout = go.Layout({
    "title":"Golden Cheetah Open Data: automatic CP determination",
    "legend": {
        "x": 0.05,
        "y": 1.1,
        "orientation": "h"
    },
    'yaxis': {'title': 'Power [Watts]'}
})


In [40]:

_df = estimate_cp(df)
data = plot_data(_df)
iplot(go.Figure(data, layout))

# Plot all CP values

In [299]:
_df = df[df['id']=='06f81197-504a-478b-a347-052d4df03044']
_df = _df.set_index('date', drop=False).sort_index()
_df[CP_COLS_ROLLING] = _df[CP_COLS].rolling('42D', min_periods=5).apply(robust_max, raw=False)

In [300]:
data = []
for col in CP_COLS_ROLLING:
    _trace = go.Scatter({
        'name': col,
        'x': _df['date'],
        'y': _df[col],
        'mode': 'markers'
    })
    data.append(_trace)
    
fig = go.Figure(data)
iplot(fig)